In [ ]:
import sys
import numpy as np
from PIL import Image

import matplotlib.pyplot as plt

import os
import glob

import cv2
import torch

# adding global TF installation to PATH 
tensorflow_path = "/usr/lib/python3.10/site-packages"
sys.path.append(tensorflow_path)
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

# Loading object detection and classification models

In [ ]:
detection_model = torch.hub.load('ultralytics/yolov5', 'custom', path='<PATH>/yolov5/runs/train/exp/weights/best.pt')
classification_model = load_model('<PATH>/logorecognition/results/logo_classification/logo_classification.model')

In [ ]:
HEIGHT = 224 # for Xception
WIDTH =  224 # for Xception
BS = 128

classification_model.get_config

def get_labels(directory):
    
    subdirs = os.listdir(directory)
    return [subdir for subdir in subdirs if os.path.isdir(os.path.join(directory, subdir))]

labels = get_labels("../data/logo_classification/test")
labels.sort() # it was sorted when training logo classification
    
def predimage(image):
    """
    Function to predict the class of an image using a pre-trained model.
    The image is first pre-processed (resized, converted to an array, and normalized)
    before making predictions using the global 'classification_model' variable.
    
    Parameters:
    ----------
    image : ndarray
        An input image, should be an array of pixel values.
    
    Returns:
    -------
    tuple
        A tuple containing the predicted label and corresponding confidence score. 
        The label is a string and the confidence score is a float.

    Notes:
    -----
    This function relies on several globally defined variables including the 'classification_model', 
    'WIDTH', 'HEIGHT', 'BS' (batch size), and 'labels' (class labels).
    """
    
    image = cv2.resize(image, (WIDTH,HEIGHT))
    image = img_to_array(image)
    image = np.expand_dims(image,axis=0)
    image /= 255 
    result = classification_model.predict(image, batch_size = BS)
    y_class = result.argmax(axis=-1)
    result = (result*100)
    result = list(np.around(np.array(result),1))
    
    return (labels[y_class[0]], result[0][y_class[0]]) # return label and confidence

# Load images

In [ ]:
images_dir = '../data/new_images/'

def get_filenames(dir):

    filenames = set()
    
    for path in glob.glob(os.path.join(dir, '*.jpg')):
        # Extract the filename
        filename = os.path.split(path)[-1]        
        filenames.add(filename)

    return filenames

images = get_filenames(images_dir)

In [ ]:
for i, image in enumerate(images):
    image_path = images_dir + image
    
    detection_model.conf = 0.7  # set confidence threshold
    detection_model.iou = 0.4  # set IoU threshold
    detection_model.augment = True  # set augment flag
    detected_logos = detection_model(image_path)
    
    bounding_boxes = []
    for x1, y1, x2, y2, conf, cls in detected_logos.xyxy[0]:
        if conf > 0.7:
            crop = cv2.imread(image_path)
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            crop = crop[y1:y2, x1:x2]  # crop to ROI
            crop = cv2.resize(crop, (HEIGHT, WIDTH))
            detection_model.augment = True
            label, confidence = predimage(crop)
            bounding_boxes.append((x1, y1, x2, y2, label, confidence))
            
    
    img = cv2.imread(image_path)
    for box in bounding_boxes:
        
        x1, y1, x2, y2, label, confidence = box
        
        if (confidence > 70):
            #draw green bounding box with label
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, label, (x1, y2-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        else:
            #draw red bounding box with no label
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
            
    cv2.imwrite(f'../results/{image}', img)